In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

import argparse
import logging
import pandas as pd
import random
import os
import numpy as np
from loggers import logger

from process import input_data, preprocess

def set_logger(log_name):
    log_obj = logger.AutoMLLog(log_name)
    log_obj.set_handler('automl_process')
    log_obj.set_formats()
    auto_logger = log_obj.addOn()
    
    auto_logger.info('logger 세팅')

In [2]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [3]:
import statsmodels.api as sm
from datetime import timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller

In [4]:
#!pip install pmdarima

In [7]:
data = pd.read_csv('storage/data/snop_prep_df_230312.csv')

In [14]:
data.columns

Index(['yymmdd', 'qty', 'rhm_avg', 'gsr_sum', 'discount', 'cust_grp_cd',
       'biz_cd', 'line_cd', 'brand_cd', '브랜드', 'SP1'],
      dtype='object')

In [12]:
pd.DataFrame(data[['SP1','브랜드']].value_counts()).to_csv('sp브랜드.csv')

In [21]:
path ='storage/data/stallion.csv'
target_var='volume'
date_var= 'date'
store_list=['sku','agency']
unit='month'
predict_n = 7

In [9]:
path ='storage/data/Walmart.csv'
target_var='Weekly_Sales'
date_var= 'Date'
store_list=['Store']
unit='week'
predict_n = 7

In [27]:
path ='storage/data/demand_forecast_dataset.csv'
target_var='sale_qty'
date_var= 'sale_dy'
store_list=['str_cd', 'prod_cd']
unit='day'
predict_n = 7

In [28]:
log_name = 'practice_tft'
set_logger(log_name)
data, var_list, num_var, obj_var = input_data.Data_load(path, log_name).read_data()
#df = preprocess.Preprocessing(log_name, data, var_list, num_var, obj_var, target_var=target_var, date_var=date_var, store_list=store_list, unit=unit).df

logger 세팅
logger 세팅
logger 세팅
logger 세팅
csv 데이터 불러오기
csv 데이터 불러오기
csv 데이터 불러오기
csv 데이터 불러오기
storage/data/demand_forecast_dataset.csv
storage/data/demand_forecast_dataset.csv
storage/data/demand_forecast_dataset.csv
storage/data/demand_forecast_dataset.csv
변수 분리 시작
변수 분리 시작
변수 분리 시작
변수 분리 시작
데이터 구성: 115050 행, 39열
데이터 구성: 115050 행, 39열
데이터 구성: 115050 행, 39열
데이터 구성: 115050 행, 39열
Memory usage of dataframe is 34.23MB
Memory usage of dataframe is 34.23MB
Memory usage of dataframe is 34.23MB
Memory usage of dataframe is 34.23MB
Memory usage after optimization is: 9.46MB
Memory usage after optimization is: 9.46MB
Memory usage after optimization is: 9.46MB
Memory usage after optimization is: 9.46MB
Decreased by 72.4%
Decreased by 72.4%
Decreased by 72.4%
Decreased by 72.4%


In [29]:
print(max(data[store_list].value_counts()))
print(min(data[store_list].value_counts()))

767
767


In [9]:
data['volume'] = data['volume'].apply(lambda x : x+1 if x ==0 else x)

In [11]:
data.head(10)

,agency,sku,volume,date,industry_volume,soda_volume,avg_max_temp,price_regular,price_actual,discount,...,labor_day,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest,discount_in_percent,timeseries
0,Agency_22,SKU_01,52.281250,2013-01-01,492612703,718394219,25.843750,1169.0,1069.0,99.7500,...,0,0,0,0,0,0,0,0,8.531250,0
1,Agency_37,SKU_04,1.000000,2013-01-01,492612703,718394219,26.500000,1852.0,1611.0,240.7500,...,0,0,0,0,0,0,0,0,13.000000,5
2,Agency_59,SKU_03,813.000000,2013-01-01,492612703,718394219,22.218750,1271.0,1197.0,73.6250,...,0,0,0,0,0,0,0,0,5.792969,9
3,Agency_11,SKU_01,316.500000,2013-01-01,492612703,718394219,25.359375,1176.0,1083.0,93.3750,...,0,0,0,0,0,0,0,0,7.941406,14
4,Agency_05,SKU_05,421.000000,2013-01-01,492612703,718394219,24.078125,1327.0,1208.0,119.1875,...,0,0,0,0,0,0,0,0,8.984375,22
5,Agency_28,SKU_06,270.750000,2013-01-01,492612703,718394219,29.968750,1099.0,950.0,148.5000,...,0,0,0,0,0,0,0,0,13.523438,32
6,Agency_05,SKU_02,876.500000,2013-01-01,492612703,718394219,24.078125,1132.0,1044.0,87.5625,...,0,0,0,0,0,0,0,0,7.738281,23
7,Agency_57,SKU_03,782.000000,2013-01-01,492612703,718394219,22.218750,1269.0,1165.0,103.5000,...,0,0,0,0,0,0,0,0,8.164062,37
8,Agency_52,SKU_04,280.500000,2013-01-01,492612703,718394219,30.593750,1728.0,1559.0,169.0000,...,0,0,0,0,0,0,0,0,9.781250,44
9,Agency_11,SKU_02,6.695312,2013-01-01,492612703,718394219,25.359375,1133.0,1047.0,85.9375,...,0,0,0,0,0,0,0,0,7.582031,18


In [18]:
if len(store_list) == 1:
    store_list = ['dummy'] + store_list
    df.loc[:,'dummy'] = 'dummy'
    
train_df = pd.DataFrame()
val_df = pd.DataFrame()
pred_df = pd.DataFrame()

for store_var_0, store_var_1 in df.drop_duplicates(store_list)[store_list].values:
    ari_df = df.loc[(df[store_list[0]]==store_var_0)&(df[store_list[1]]==store_var_1), :]               
    ari_df.loc[:, 'ds'] = ari_df[date_var]
    ari_df.loc[:, 'y'] = ari_df[target_var]        

    predict_size = predict_n
    ari_train = ari_df.iloc[:-predict_size, :]
    ari_var = ari_df.iloc[-predict_size:, :]
    
    #auto arima best parameter pick
    ari = auto_arima(y = ari_train['y'].values, d = 1, start_p = 0, max_p = 3, start_q = 0 
                      , max_q = 3, m = 1, seasonal = False , stepwise = True, trace=True)
    
    #arima fit 후 ari_var에 예측값 대입
    ari.fit(ari_train['y'].values)
    val_preds = ari.predict(n_periods = predict_size)
    ari_var.loc[:, 'yhat'] = val_preds
    
    #train
    train= ari_train[['y','ds']]
    train.loc[:, store_list[0]] = store_var_0
    train.loc[:, store_list[1]] = store_var_1
    
    train_df = pd.concat([train_df, train], axis=0)

    #valid
    val_preds_df = ari_var[['ds','yhat']]
    val_preds_df.loc[:, store_list[0]] = store_var_0
    val_preds_df.loc[:, store_list[1]] = store_var_1
    
    val_df = pd.concat([val_df, val_preds_df], axis=0)
    
    #predict_date 생성 후 예측 predict_df생성
    last_date = ari_df[date_var].iloc[-1:].tolist()[0]
    
    if unit =='day':
        predict_date = [last_date + timedelta(days=i) for i in range(1, predict_n+1)] #weeks, days 변경 가능
    elif unit == 'week':
        predict_date = [last_date + timedelta(days=7*i) for i in range(1, predict_n+1)] #weeks, days 변경 가능
    elif unit == 'month':
        predict_date = [last_date + timedelta(days=30*i) for i in range(1, predict_n+1)] #weeks, days 변경 가능
        
    test_df = pd.DataFrame({'ds':predict_date})
    ari.fit(ari_df['y'].values)
    preds = ari.predict(n_periods = predict_size)
    test_df.loc[:, 'yhat'] = preds
    test_df.loc[:, store_list[0]] = store_var_0
    test_df.loc[:, store_list[1]] = store_var_1
    
    pred_df = pd.concat([pred_df, test_df], axis=0)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=526.142, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=511.267, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=524.196, Time=0.00 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=506.453, Time=0.03 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=507.510, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.19 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=504.855, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=509.474, Time=0.01 sec
 ARIMA(3,1,0)(0,0,0)[0]             : AIC=505.998, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=503.114, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=501.351, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=499.353, Time=0.01 se

In [19]:
train_df

,y,ds,sku,agency
0,52.28125,2013-01-01,SKU_01,Agency_22
1,62.53125,2013-02-01,SKU_01,Agency_22
2,74.18750,2013-03-01,SKU_01,Agency_22
3,89.43750,2013-04-01,SKU_01,Agency_22
4,79.18750,2013-05-01,SKU_01,Agency_22
...,...,...,...,...
20988,964.50000,2017-01-01,SKU_03,Agency_42
20989,1444.00000,2017-02-01,SKU_03,Agency_42
20990,1640.00000,2017-03-01,SKU_03,Agency_42
20991,1505.00000,2017-04-01,SKU_03,Agency_42


In [20]:
val_df

,ds,yhat,sku,agency
53,2017-06-01,111.120958,SKU_01,Agency_22
54,2017-07-01,111.120958,SKU_01,Agency_22
55,2017-08-01,111.120958,SKU_01,Agency_22
56,2017-09-01,111.120958,SKU_01,Agency_22
57,2017-10-01,111.120958,SKU_01,Agency_22
...,...,...,...,...
20995,2017-08-01,1618.303545,SKU_03,Agency_42
20996,2017-09-01,1636.470441,SKU_03,Agency_42
20997,2017-10-01,1654.637336,SKU_03,Agency_42
20998,2017-11-01,1672.804231,SKU_03,Agency_42


In [21]:
pred_df

,ds,yhat,sku,agency
0,2017-12-31,109.496920,SKU_01,Agency_22
1,2018-01-30,109.496920,SKU_01,Agency_22
2,2018-03-01,109.496920,SKU_01,Agency_22
3,2018-03-31,109.496920,SKU_01,Agency_22
4,2018-04-30,109.496920,SKU_01,Agency_22
...,...,...,...,...
2,2018-03-01,1852.227569,SKU_03,Agency_42
3,2018-03-31,1872.102222,SKU_03,Agency_42
4,2018-04-30,1891.976875,SKU_03,Agency_42
5,2018-05-30,1911.851529,SKU_03,Agency_42


In [38]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.api import Holt

In [39]:
if len(store_list) == 1:
    store_list = ['dummy'] + store_list
    df.loc[:,'dummy'] = 'dummy'
    
train_df = pd.DataFrame()
val_df = pd.DataFrame()
pred_df = pd.DataFrame()

for store_var_0, store_var_1 in df.drop_duplicates(store_list)[store_list].values:
    ets_df = df.loc[(df[store_list[0]]==store_var_0)&(df[store_list[1]]==store_var_1), :]               
    ets_df.loc[:, 'ds'] = ets_df[date_var]
    ets_df.loc[:, 'y'] = ets_df[target_var]        

    predict_size = predict_n
    ets_train = ets_df.iloc[:-predict_size, :]
    ets_var = ets_df.iloc[-predict_size:, :]
    
    #auto arima best parameter pick
    ets = Holt(ets_train['y'].values, exponential=True, initialization_method="estimated").fit(
            smoothing_level=0.8, smoothing_trend=0.2, optimized=False)
    
    #arima fit 후 ari_var에 예측값 대입
    
    val_preds = ets.forecast(predict_size)
    ets_var.loc[:, 'yhat'] = val_preds
    
    #train
    train= ets_train[['y','ds']]
    train.loc[:, store_list[0]] = store_var_0
    train.loc[:, store_list[1]] = store_var_1
    
    train_df = pd.concat([train_df, train], axis=0)

    #valid
    val_preds_df = ets_var[['ds','yhat']]
    val_preds_df.loc[:, store_list[0]] = store_var_0
    val_preds_df.loc[:, store_list[1]] = store_var_1
    
    val_df = pd.concat([val_df, val_preds_df], axis=0)
    
    #predict_date 생성 후 예측 predict_df생성
    last_date = ets_df[date_var].iloc[-1:].tolist()[0]
    
    if unit =='day':
        predict_date = [last_date + timedelta(days=i) for i in range(1, predict_n+1)] #weeks, days 변경 가능
    elif unit == 'week':
        predict_date = [last_date + timedelta(days=7*i) for i in range(1, predict_n+1)] #weeks, days 변경 가능
    elif unit == 'month':
        predict_date = [last_date + timedelta(days=30*i) for i in range(1, predict_n+1)] #weeks, days 변경 가능
        
    test_df = pd.DataFrame({'ds':predict_date})
    ets = Holt(ets_df['y'].values, exponential=True, initialization_method="estimated").fit(
            smoothing_level=0.8, smoothing_trend=0.2, optimized=False)
    
    preds = ets.forecast(predict_size)
    test_df.loc[:, 'yhat'] = preds
    test_df.loc[:, store_list[0]] = store_var_0
    test_df.loc[:, store_list[1]] = store_var_1
    
    pred_df = pd.concat([pred_df, test_df], axis=0)

In [43]:
val_df

,ds,yhat,dummy,Store
136,2012-09-28,1.503448e+06,dummy,1
137,2012-10-02,1.493173e+06,dummy,1
138,2012-10-08,1.482969e+06,dummy,1
139,2012-10-19,1.472834e+06,dummy,1
140,2012-10-26,1.462769e+06,dummy,1
...,...,...,...,...
6430,2012-10-08,7.087306e+05,dummy,45
6431,2012-10-19,7.048536e+05,dummy,45
6432,2012-10-26,7.009979e+05,dummy,45
6433,2012-11-05,6.971632e+05,dummy,45


In [ ]:
x`